In [2]:
#### Importación de librerias ####

from tkinter import *
from tkinter import filedialog
import PIL.Image, PIL.ImageTk
import cv2
from PIL import Image, ImageTk
import numpy as np
import os
from collections import deque
import pickle
import face_recognition
from collections import deque

##### Inicio Clase #####

class videoGUI:
    
    # Función constructora
    def __init__(self, window, window_title): 
                        
        # configuración de ventana principal
        self.window = window
        self.window.title(window_title)
        self.window.resizable(False, False)
        self.window.protocol('WM_DELETE_WINDOW', self.doSomething)  # root is your root window
        
        # Generacion de los 2 frame (Menu / vista 1 y 2)
        top_frame = Frame(self.window,relief="raised", borderwidth=2)
        top_frame.pack(side=RIGHT, pady=5,padx=5)
        bottom_frame = Frame(self.window,relief="raised", borderwidth=2)
        bottom_frame.pack(side=LEFT, pady=5 , padx=5)

        self.pause = False   # Controlador de estado de video (pause/play)
        
        self.canvas = Canvas(top_frame) # Agrupación
        self.canvas.pack()
        
        # Etiquetas
        self.etiq1 = Label(bottom_frame, text="Menu:")
        self.etiq1.grid(column=1, row=0,sticky = W)
        
        # Checkbutton
        self.Equipo_1 = IntVar()    # 1 si, 0 no
        self.Equipo_2 = IntVar()   # 1 si, 0 no
        self.Rostros = IntVar()    # 1 si, 0 no
        self.Lineas = IntVar()    # 1 si, 0 no
        

        self.cb_Equipo_1 = Checkbutton(bottom_frame, text = "Equipo Azul ", variable=self.Equipo_1, onvalue=1, offvalue=0) 
        self.cb_Equipo_2 = Checkbutton(bottom_frame, text = "Equipo Rojo", variable=self.Equipo_2, onvalue=1, offvalue=0)
        
        self.cb_Rostros = Checkbutton(bottom_frame, text = "Reconocimiento", variable=self.Rostros, onvalue=1, offvalue=0)
        self.cb_Lineas = Checkbutton(bottom_frame, text = "Lineas", variable=self.Lineas, onvalue=1, offvalue=0)
        
        # Select Button
        self.btn_select=Button(bottom_frame, text="Seleccione un archivo de video", width=25, command=self.open_file)
        self.btn_select.grid(row=1, column=1, columnspan=2, pady=2)

        # Play Button
        self.btn_play=Button(bottom_frame, text="Play", width=25, command=self.play_video)
        self.btn_play.grid(row=2, column=1, columnspan=2, pady=2)

        # Pause Button
        self.btn_pause=Button(bottom_frame, text="Pause", width=25, command=self.pause_video)
        self.btn_pause.grid(row=3, column=1, columnspan=2, pady=2)

        # Resume Button
        self.btn_resume=Button(bottom_frame, text="Continuar", width=25, command=self.resume_video)
        self.btn_resume.grid(row=4, column=1, columnspan=2, pady=2)
        
        
        # Etiqueta
        
        self.etiq1 = Label(bottom_frame, text="Detectar:")
        self.etiq1.grid(column=1, row=8,sticky = W)
        
        self.cb_Equipo_1.grid(row = 9, column = 1, sticky="W") 
        self.cb_Equipo_2.grid(row = 10, column = 1, sticky="W") 
        self.cb_Rostros.grid(row = 11, column = 1, sticky="W") 
        self.cb_Lineas.grid(row = 12, column = 1, sticky="W") 
        
        self.pickle_in = open("model_trained.p","rb")
        self.model = pickle.load(self.pickle_in)
        self.umbral = 0.5        

        self.reduccion = 5 #Con un 3, la imagen se reducirá a 1/3 del tamaño original
        self.font = cv2.FONT_HERSHEY_SIMPLEX
        self.encodings_conocidos,self.nombres_conocidos = self.generar_encode()
        
        # Configuración de ingreso de video
        self.delay = 25   # ms
        
        self.anterior = 0
        self.nuevo = 0
        self.centros = deque(maxlen=20)

        ####################### controladores de opciones #########################
        self.Jugadores_rostro = False
        self.Gol = False
        self.velocidad = False
        
        self.window.mainloop()
        
    def generar_encode(self):
        #Cargar la imagen de ejemplo con nuestro rostro:
        imagen_Oliver = face_recognition.load_image_file("img/Olivier Giroud.jpg")
        imagen_Andres = face_recognition.load_image_file("img/Andres Cunha.jpeg")
        imagen_Griezmann = face_recognition.load_image_file("img/Griezmann.jpg")
        #Extraer los 'encodings' que caracterizan nuestro rostro:
        Oliver_encodings = face_recognition.face_encodings(imagen_Oliver)[0]
        Andres_encodings = face_recognition.face_encodings(imagen_Andres)[0]
        Griezmann_encodings = face_recognition.face_encodings(imagen_Griezmann)[0]
        #Definir un array con los encodings y nuestro nombre:
        encodings_conocidos = [
            Oliver_encodings,
            Andres_encodings,
            Griezmann_encodings
        ]
        nombres_conocidos = [
            "Olivier Giroud",
            "Andres Cunha",
            "Antoine Griezmann"
        ]
        return encodings_conocidos,nombres_conocidos
    
    # Función para abrir archivo
    def open_file(self): 

        self.pause = True

        self.filename = filedialog.askopenfilename(title="Select file", filetypes=(("MP4 files", "*.mp4"),
                                                                                         ("WMV files", "*.wmv"), ("AVI files", "*.avi")))
        print(self.filename)

        # Open the video file
        self.cap = cv2.VideoCapture(self.filename)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))        
        self.canvas.config(width = 650 , height = 400)
                
        vector=self.filename.split("/")
        vector[-1]
        ##### Selector de opciones ############
        
        if(vector[-1] == 'Reconocimiento - Juego.mp4'):
            
            self.habilitarJugadores()
            self.Jugadores_rostro=True
            self.Gol=False
            self.velocidad=False
            
        elif(vector[-1] == 'Gol.mp4'):
        
            self.deshabilitarJugadores()
            self.Jugadores_rostro=False
            self.Gol=True
            self.velocidad=False
            #print('Funcion no implementada')
            
        elif(vector[-1] == 'Velocidad.mp4'):
        
            self.deshabilitarJugadores()
            self.Jugadores_rostro=False
            self.Gol=False            
            self.velocidad=True
            #print('Funcion no implementada')
            
        else:
            print("Todas Funciones deshabilitadas")
            self.deshabilitarJugadores()
            self.Jugadores_rostro=False
            self.Gol=False
            
        print("Video Cargado")
        self.play_video() # Llamada a la funcion de inicio
    
    def habilitarJugadores(self):
        
        self.cb_Equipo_1.config(state=NORMAL)
        self.cb_Equipo_2.config(state=NORMAL)
        self.cb_Rostros.config(state=NORMAL)
        
        print('Funciones de deteccion habilitadas')
        
    def deshabilitarJugadores(self):
        
        self.cb_Equipo_1.config(state=DISABLED)
        self.cb_Equipo_2.config(state=DISABLED)
        self.cb_Rostros.config(state=DISABLED)
        print('Funciones de deteccion habilitadas')
    
    def preProcessing(self,img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.equalizeHist(img)
        img = img/255    
        return img
    
    # Obtener 1 Frame
    def get_frame(self):   

        try:

            if self.cap.isOpened():
                ret, frame = self.cap.read()
                return (ret, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        except:
            print("video terminado")
            #messagebox.showerror(title='Video Terminado o no encontrado ', message='Ingresar un archivo valido o cambiar video')
            
    # Función para abrir camara
    def abrir_camara(self): 
        
        self.pause = False
        self.cap = cv2.VideoCapture(0)
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.canvas.config(width = 650 , height = 670)
        self.btn_play.config(state=DISABLED)
        self.btn_resume.config(state=DISABLED)
        self.play_video() # Llamada a la funcion de inicio
        
    # Función principal
    def play_video(self): 
        
        # Obtiene 1 frame del video, obtendra el siguiente automaticamente
        ret, image = self.get_frame()
        
        if ret:
            image = cv2.resize(image, (569, 320), interpolation = cv2.INTER_CUBIC) # Cambiar tamaño

            if(self.Jugadores_rostro == True):
                
                if (self.Rostros.get()):
                    
                    image = cv2.resize(image,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                    reconocimiento = image.copy()
                    #image = cv2.resize(image, (569, 320), interpolation = cv2.INTER_CUBIC) # Cambiar tamaño
                    #image = cv2.resize(image,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                    #reconocimiento = image.copy()

                    #original_image = image.copy()
                    #original_image = original_image[:, :, ::-1]

                    ########################## RECONOCIMIENTO DE ROSTROS ####################
                    #Definimos algunos arrays y variables:
                    loc_rostros = [] #Localizacion de los rostros en la imagen
                    encodings_rostros = [] #Encodings de los rostros
                    nombres_rostros = [] #Nombre de la persona de cada rostro
                    nombre = "" #Variable para almacenar el nombre
                    ##################### RECONOCIMIENTO FACIAL ########################
                    img_2=reconocimiento.copy()

                    #La imagen está en el espacio de color BGR, habitual de OpenCV. Hay que convertirla a RGB:
                    img_rgb = reconocimiento[:, :, ::-1]

                    #Reducimos el tamaño de la imagen para que sea más rápida de procesar:
                    img_rgb = cv2.resize(img_rgb, (0, 0), fx=1.0/self.reduccion, fy=1.0/self.reduccion)

                    #cv2.imshow('image',img_rgb)
                    #Localizamos cada rostro de la imagen y extraemos sus encodings:
                    loc_rostros = face_recognition.face_locations(img_rgb)

                    #print (loc_rostros)
                    encodings_rostros = face_recognition.face_encodings(img_rgb, loc_rostros)
                    #print(encodings_rostros)

                    for encoding in encodings_rostros:
                        #Buscamos si hay alguna coincidencia con algún encoding conocido:
                        coincidencias = face_recognition.compare_faces(self.encodings_conocidos, encoding)
                        #print("entro")
                        #El array 'coincidencias' es ahora un array de booleanos. Si contiene algun 'True', es que ha habido alguna coincidencia:
                        if True in coincidencias:
                            nombre = self.nombres_conocidos[coincidencias.index(True)]

                        #Si no hay ningún 'True' en el array 'coincidencias', no se ha podido identificar el rostro:
                        else:
                            nombre = "???"

                        #Añadir el nombre de la persona identificada en el array de nombres:
                        nombres_rostros.append(nombre)

                    #Dibujamos un recuadro rojo alrededor de los rostros desconocidos, y uno verde alrededor de los conocidos:
                    for (top, right, bottom, left), nombre in zip(loc_rostros, nombres_rostros):

                        #Deshacemos la reducción de tamaño para tener las coordenadas de la imagen original:
                        top = top*self.reduccion
                        right = right*self.reduccion
                        bottom = bottom*self.reduccion
                        left = left*self.reduccion
                        #Cambiar de color según si se ha identificado el rostro:
                        if nombre != "???":
                            color = (0,255,0)
                        else:
                            color = (0,0,255)

                        #Dibujar un rectángulo alrededor de cada rostro identificado, y escribir el nombre:
                        cv2.rectangle(image, (left, top), (right, bottom), color, 2)
                        cv2.rectangle(image, (left, bottom - 20), (right, bottom), color, -1)
                        cv2.putText(image, nombre, (left, bottom - 6), self.font, 0.6, (0,0,0), 1) 

                    #cv2.imshow('Img_2',mask)

                ######################################################################################################################
                
                original_image = image.copy()
                original_image = original_image[:, :, ::-1]
                
                #converting into hsv image
                hsv = cv2.cvtColor(original_image,cv2.COLOR_BGR2HSV)

    #             #green range
    #             lower_green = np.array([40,40, 40])
    #             upper_green = np.array([70, 255, 255])
    #             #blue range
    #             lower_blue = np.array([110,50,50])
    #             upper_blue = np.array([130,255,255])
    #             #Red range
    #             lower_red = np.array([0,31,255])
    #             upper_red = np.array([176,255,255])
    #             #white range
    #             lower_white = np.array([0,0,0])
    #             upper_white = np.array([0,0,255])

                #cv2.imshow('hsv',hsv)
                #green range pasto
                lower_green = np.array([40,40, 40])
                upper_green = np.array([70, 255, 255])

                #blue range everton
                lower_blue = np.array([97,74,18])
                upper_blue = np.array([117,196,159])

                #Red range liverpool 
                lower_red = np.array([0,100,20])
                upper_red = np.array([15,255,255])

                #white range balon y lineas
                lower_white = np.array([0,0,0])
                upper_white = np.array([0,0,255])


                #Define a mask ranging from lower to uppper
                mask = cv2.inRange(hsv, lower_green, upper_green)
                #Do masking
                res = cv2.bitwise_and(image, image, mask=mask)


                #convert to hsv to gray
                res_bgr = cv2.cvtColor(res,cv2.COLOR_HSV2BGR)
                res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)

                #Defining a kernel to do morphological operation in threshold image to 
                #get better output.
                kernel = np.ones((13,13),np.uint8)
                thresh = cv2.threshold(res_gray,127,255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
                thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

                #find contours in threshold image     
                im2,contours,hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)


                #cv2.imshow('Match im2',mask)

                for c in contours:
                    x,y,w,h = cv2.boundingRect(c)
                    #Detect players

                    if(h>=(1.5)*w):
                        #print(x,y,w,h)
                        if(w>10 and h>= 10):

                            classIndex = None
                            probVal = None
                            player_img = None
                            player_img = original_image[y:y+h,x:x+w]

                            #cv2.imshow('Match im2',player_img)

                            player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)

                            #If player has blue jersy
                            mask1 = cv2.inRange(player_hsv, lower_blue, upper_blue)
                            res1 = cv2.bitwise_and(player_img, player_img, mask=mask1)
                            res1 = cv2.cvtColor(res1,cv2.COLOR_HSV2BGR)
                            res1 = cv2.cvtColor(res1,cv2.COLOR_BGR2GRAY)
                            nzCount = cv2.countNonZero(res1)
                            #If player has red jersy
                            mask2 = cv2.inRange(player_hsv, lower_red, upper_red)
                            res2 = cv2.bitwise_and(player_img, player_img, mask=mask2)
                            res2 = cv2.cvtColor(res2,cv2.COLOR_HSV2BGR)
                            res2 = cv2.cvtColor(res2,cv2.COLOR_BGR2GRAY)
                            nzCountred = cv2.countNonZero(res2)
                            mensaje = ""
                            #cv2.imshow('Ms 1',mask1)
                            #cv2.imshow('Ms 2',mask2)
                            
                            if(self.Equipo_1.get()):


                                if(nzCount >= 20):
                                    #Mark blue jersy players as france
                                    img = np.asarray(player_img)
                                    img = cv2.resize(img, (32,32))
                                    img = self.preProcessing(img)
                                    img = img.reshape(1,32,32,1)
                                    classIndex = int (self.model.predict_classes(img))
                                    predictions = self.model.predict(img)
                                    probVal= np.amax(predictions)

                                    if(probVal > self.umbral ):
                                        mensaje = 'Francia '+ str(classIndex)
                                    else:
                                        mensaje = 'Francia'

                                    cv2.putText(image, mensaje, (x-2, y-2), self.font, 0.8, (0,0,255), 2, cv2.LINE_AA)
                                    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)

                                else:
                                    pass
                            
                            if(self.Equipo_2.get()):
                            
                                if(nzCountred>=20):
                                    #Mark red jersy players as belgium
                                    img = np.asarray(player_img)
                                    img = cv2.resize(img, (32,32))
                                    img = self.preProcessing(img)
                                    img = img.reshape(1,32,32,1)

            #                        mensaje = pytesseract.image_to_string(player_img, config=custom_config)
                                    classIndex = int (self.model.predict_classes(img))
                                    predictions = self.model.predict(img)
                                    probVal= np.amax(predictions)
                                    if(probVal > self.umbral):
                                        mensaje = 'Belgica '+ str(classIndex)
                                    else:
                                        mensaje = 'Belgica'
                                    cv2.putText(image, mensaje, (x-2, y-2), self.font, 0.8, (255,0,0), 2, cv2.LINE_AA)
                                    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)

                                else:
                                    pass
                            #cv2.imshow('Player',player_img)   
                    
                    
                    if((h>=1 and w>=1) and (h<=30 and w<=30)):
                        ball_img = original_image[y:y+h,x:x+w]
                        ball_hsv = cv2.cvtColor(ball_img,cv2.COLOR_BGR2HSV)
                        #white ball  detection
                        mask_b = cv2.inRange(ball_hsv, lower_white, upper_white)
                        res_b = cv2.bitwise_and(ball_img, ball_img, mask=mask_b)
                        res_b = cv2.cvtColor(res_b,cv2.COLOR_HSV2BGR)
                        res_b = cv2.cvtColor(res_b,cv2.COLOR_BGR2GRAY)
                        nzCount_b = cv2.countNonZero(res_b)


                        if(nzCount_b >= 3):
                            # detect football
                            cv2.putText(image, 'balon', (x-2, y-2), self.font, 0.8, (0,255,0), 2, cv2.LINE_AA)
                            cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
                image = cv2.resize(image, (569, 320), interpolation = cv2.INTER_CUBIC) # Cambiar tamaño
                #cv2.imshow('Match Detection',image)
                img = Image.fromarray(image)                                         
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(43,40, image = self.photo, anchor = NW)
                
            else:
                img = Image.fromarray(image)                                         
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(43,40, image = self.photo, anchor = NW)
        
            if(self.Gol==True):
                
                puntoAlejadoCirculo=0
                puntoAlejadoLinea=0
                #image = cv2.resize(image,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                #original_image = image.copy()
                gray = cv2.medianBlur(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),5)
                #cv2.imshow('Match im2',gray)

                #detect lines
        #        high_threshold, thresh_im = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        #        low_threshold = 0.5*high_threshold
                low_threshold = 100
                high_threshold = 300
                edges = cv2.Canny(gray, low_threshold, high_threshold)

                rho = 1  # distance resolution in pixels of the Hough grid
                theta = np.pi / 180  # angular resolution in radians of the Hough grid
                threshold = 10  # minimum number of votes (intersections in Hough grid cell)
                min_line_length = 140  # minimum number of pixels making up a line
                max_line_gap = 20  # maximum gap in pixels between connectable line segments
                line_image = np.copy(image) * 0  # creating a blank to draw lines on

                # Run Hough on edge detected image
                # Output "lines" is an array containing endpoints of detected line segments
                lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                                    min_line_length, max_line_gap)

                if lines is not None:
                    for line in lines:
                        for x1, y1, x2, y2 in line:
                            if(x1 == x2):
                                puntoAlejadoLinea= x1
                                cv2.line(image, (x1, y1), (x2, y2), (0,0,0), 4)
                                cv2.putText(image, str(puntoAlejadoLinea), (puntoAlejadoLinea, y1), self.font, 0.6, (0,0,0), 3) 

                #cv2.imshow('Match lines',line_image)


                # detect circles in the image
                circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT,0.3,180,param1=140,param2=90,minRadius=15,maxRadius=140)

                # ensure at least some circles were found
                if circles is not None:
                    # convert the (x, y) coordinates and radius of the circles to integers
                    circles = np.round(circles[0, :]).astype("int")
                    # loop over the (x, y) coordinates and radius of the circles
                    for (x, y, r) in circles:
                        # draw the circle in the output image, then draw a rectangle
                        # corresponding to the center of the circle
                        cv2.circle(image, (x, y), r, (0,0,0), 4)
                        cv2.rectangle(image, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
                        puntoAlejadoCirculo= x+r
                        cv2.putText(image, str(puntoAlejadoCirculo), (puntoAlejadoCirculo, y), self.font, 0.6, (0,0,0), 3) 

                if(puntoAlejadoCirculo!=0 and puntoAlejadoLinea != 0):
                    if(puntoAlejadoCirculo>puntoAlejadoLinea):
                        cv2.putText(image, "No GOL", (80, 100), self.font, 4, (255,255,255), 3)
                    else:
                        cv2.putText(image, "GOL", (80, 100), self.font, 4, (255,255,255), 3)
                #cv2.imshow('Match Detection',image)
                img = Image.fromarray(image)                                         
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(43,40, image = self.photo, anchor = NW)
                
            if(self.velocidad==True):
                
                gray = cv2.medianBlur(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY),5)
                low_threshold = 100
                high_threshold = 600
                edges = cv2.Canny(gray, low_threshold, high_threshold)

                #cv2.imshow('Match ',edges)

                #find contours in threshold image     
                im2,contours,hierarchy = cv2.findContours(edges,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

                for c in contours:
                    x,y,w,h = cv2.boundingRect(c)
                    #print( x,y,w,h)
                    #cv2.putText(image, str(x)+' '+str(y), (x, y-40), self.font, 0.6, (0,0,0), 1) 
                    #Detect players
                    if((w>10 and h>= 10) and (w<70 and h <70) and (y >180 and y<250) and(x>125 and x<450)):
                        cv2.rectangle(image, (x, y), (x, y), (0, 128, 255), -1)
                        #cv2.putText(image, str(x)+' '+str(y), (x, y-40), self.font, 0.6, (0,0,0), 1) 
                        self.anterior = self.nuevo
                        self.nuevo = x
                        
                        self.centros.appendleft((x, y+15))
                        
                        if self.anterior !=0:
                            distancia = abs(self.nuevo - self.anterior)
                            #print(x,y)
                            velocidad = (distancia*3/325) / (1/30)
                            cv2.putText(image, str(round(velocidad, 2)), (x, y-20), self.font, 0.6, (0,0,0), 1)
                            cv2.circle(image,(x+12,y+15), 15, (0,0,255), 1)
                            
                            break
                            
                for centro in self.centros:
                    cv2.circle(image, centro, 1, (0, 0, 255), -1)
                
                #cv2.imshow('Match Detection',image)
                img = Image.fromarray(image)                                         
                self.photo = ImageTk.PhotoImage(image=img)
                self.canvas.create_image(43,40, image = self.photo, anchor = NW)            
                
            if(self.Lineas.get()):
                
                image = cv2.resize(image,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
                #Caracteristicas del color a reconocer  
                low_threshold = 50
                high_threshold = 150
                kernel_size = 5  

                rho = 1  #distancia en pixeles de la cuadricula de Hough
                theta = np.pi / 180  # resolucion angular
                threshold = 15 #inserciones en la celda
                min_line_length = 190 #numeor minimo de pixeles de c/linea
                max_line_gap = 7 #espacio maximo entre pixeles en las lineas
                
                line_image = np.copy(image)* 0
                
                frame_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                blur_gray = cv2.GaussianBlur(frame_gray,(kernel_size, kernel_size),0)
                #Aplicamos las caracteristicas del reconocimiento de bordes de campo
                edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
                
                lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),min_line_length, max_line_gap)
                
                if lines is not None:
                
                    #Dibujamos cada linea segun nos rebote el metodo lines
                    for line in lines:
                            for x1,y1,x2,y2 in line:
                                cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),6)

                    lines_edges = cv2.addWeighted(image, 1, line_image, 1, 0)

                    #Mostramos la imagen original con la marca de las lineas 
                    #cv2.imshow('Original', imagen)
                    lines_edges = cv2.resize(lines_edges, (569, 320), interpolation = cv2.INTER_CUBIC) # Cambiar tamaño
                    img = Image.fromarray(lines_edges)                                         
                    self.photo = ImageTk.PhotoImage(image=img)
                    self.canvas.create_image(43,40, image = self.photo, anchor = NW)
                    #cv2.imshow('Lineas en el campo', lines_edges)
                    
                else:
                    image = cv2.resize(image, (569, 320), interpolation = cv2.INTER_CUBIC) # Cambiar tamaño
                    img = Image.fromarray(image)                                         
                    self.photo = ImageTk.PhotoImage(image=img)
                    self.canvas.create_image(43,40, image = self.photo, anchor = NW)
                    
        
        else: # else primer if
            print('no video')
            #vidcap.set(cv2.CAP_PROP_POS_FRAMES, 0) 
            
            
            
#         #Verificar pausa y velocidad de obtención de video
        if not self.pause:
            self.window.after(self.delay, self.play_video)

    # Función de pausa de video
    def pause_video(self):
        self.pause = True

    # Función para continuar video   
    def resume_video(self):
        self.pause=False
        self.play_video()


    # Terminar procesos
    def __del__(self):
        if self.cap.isOpened():
            self.cap.release()
            self.salida.release()
            cv2.release()
            cv2.destroyAllWindows()
    

    def doSomething(self):
        try:
            if self.cap.isOpened():
                self.cap.release()
                #self.salida.release()
                cv2.destroyAllWindows()
            self.window.destroy()
        except:
            cv2.destroyAllWindows()
            self.window.destroy()         

##### Fin Clase #####

# LLamada al contructor de la clase
videoGUI(Tk(), "Deteccion de objetos")

C:/Users/USUARIO/Desktop/Trabajo final de Graficos/Reconocimiento - Juego.mp4
Funciones de deteccion habilitadas
Video Cargado
video terminado


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\USUARIO\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\USUARIO\Anaconda3\lib\tkinter\__init__.py", line 749, in callit
    func(*args)
  File "<ipython-input-2-7b00e1e85e43>", line 228, in play_video
    ret, image = self.get_frame()
TypeError: cannot unpack non-iterable NoneType object


C:/Users/USUARIO/Desktop/Trabajo final de Graficos/Gol.mp4
Funciones de deteccion habilitadas
Video Cargado
C:/Users/USUARIO/Desktop/Trabajo final de Graficos/Velocidad.mp4
Funciones de deteccion habilitadas
Video Cargado

Todas Funciones deshabilitadas
Funciones de deteccion habilitadas
Video Cargado


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\USUARIO\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-7b00e1e85e43>", line 176, in open_file
    self.play_video() # Llamada a la funcion de inicio
  File "<ipython-input-2-7b00e1e85e43>", line 228, in play_video
    ret, image = self.get_frame()
TypeError: cannot unpack non-iterable NoneType object


C:/Users/USUARIO/Desktop/Trabajo final de Graficos/canchaPro.mp4
Todas Funciones deshabilitadas
Funciones de deteccion habilitadas
Video Cargado

Todas Funciones deshabilitadas
Funciones de deteccion habilitadas
Video Cargado


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\USUARIO\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-2-7b00e1e85e43>", line 176, in open_file
    self.play_video() # Llamada a la funcion de inicio
  File "<ipython-input-2-7b00e1e85e43>", line 228, in play_video
    ret, image = self.get_frame()
TypeError: cannot unpack non-iterable NoneType object
